In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.svm import SVC
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import confusion_matrix
from keras.layers import Dense, Activation, Dropout
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import random

from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import LeaveOneGroupOut
import joblib

In [2]:
df1 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Audio/raw_audio_features_studentlife.csv')
df1 = df1.rename(columns={'Date': 'date'})
df1 = df1.drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('/Users/ohidabinteamin/Documents/Stress Prediction Project Three Datasets/StudentLife/week 01/Stress/recreating_dailystress_features.csv')
df2 = df2.drop('Unnamed: 0', axis=1)

In [3]:
df = pd.merge(df1, df2, on=['uid', 'date'])
print(df.columns)

df = df.sort_values(by='date')

Index(['uid', 'date', 'morning_noise_duration', 'morning_silent_duration',
       'morning_voice_duration', 'afternoon_noise_duration',
       'afternoon_silent_duration', 'afternoon_voice_duration',
       'evening_noise_duration', 'evening_silent_duration',
       'evening_voice_duration', 'night_noise_duration',
       'night_silent_duration', 'night_voice_duration', 'stress_ratings'],
      dtype='object')


In [4]:
df.isnull().sum()

uid                          0
date                         0
morning_noise_duration       0
morning_silent_duration      0
morning_voice_duration       0
afternoon_noise_duration     0
afternoon_silent_duration    0
afternoon_voice_duration     0
evening_noise_duration       0
evening_silent_duration      0
evening_voice_duration       0
night_noise_duration         0
night_silent_duration        0
night_voice_duration         0
stress_ratings               0
dtype: int64

In [5]:
df = df.dropna()
print(len(df))

1216


In [6]:
df.columns

Index(['uid', 'date', 'morning_noise_duration', 'morning_silent_duration',
       'morning_voice_duration', 'afternoon_noise_duration',
       'afternoon_silent_duration', 'afternoon_voice_duration',
       'evening_noise_duration', 'evening_silent_duration',
       'evening_voice_duration', 'night_noise_duration',
       'night_silent_duration', 'night_voice_duration', 'stress_ratings'],
      dtype='object')

In [7]:
df['stress_ratings'].value_counts()

stress_ratings
medium stress    484
low stress       366
high stress      366
Name: count, dtype: int64

In [8]:
binary_lh_data = df[df['stress_ratings'].isin(['low stress', 'high stress'])]

In [9]:
X = binary_lh_data.drop(columns=['stress_ratings', 'uid', 'date'])
y = binary_lh_data['stress_ratings']
groups = binary_lh_data['uid']

stress_map = {'low stress': 0, 'high stress': 1}
y_encoded = y.map(stress_map).values 

In [10]:
logo = LeaveOneGroupOut()

best_thresholds = []
balanced_accs = []
auc_scores = []

In [11]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

In [12]:
num_splits = len(np.unique(groups))
logo = LeaveOneGroupOut()

In [13]:
num_splits = len(np.unique(groups))
class_weights = compute_class_weight('balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weight_dict = dict(enumerate(class_weights))

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
    'class_weight': [class_weight_dict]
}

best_thresholds = []
balanced_accs = []
auc_scores = []
auprc_scores = []
best_params_list = []

In [14]:
with tqdm(total=num_splits, desc="LOSO CV Progress", unit="fold", bar_format="{l_bar}{bar} | {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {percentage:3.0f}%") as pbar:
    for train_idx, test_idx in logo.split(X, y_encoded, groups=groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]  

        print(f"Train shape: {X_train.shape}, {y_train.shape}")

        scaler = StandardScaler()
        X_train_normalized = scaler.fit_transform(X_train)
        X_test_normalized = scaler.transform(X_test)

        print(f"Normalized shapes: Train {X_train_normalized.shape}, Test {X_test_normalized.shape}")

        print('Performing Grid Search for SVM')
        grid_search = GridSearchCV(
            SVC(kernel='rbf', probability=True),
            param_grid,
            cv=5,
            scoring='balanced_accuracy',
            n_jobs=-1 
        )

        grid_search.fit(X_train_normalized, y_train)
        model_svm_rbf = grid_search.best_estimator_
        
        print("Best parameters:", grid_search.best_params_)
        print("Best cross-validation score:", grid_search.best_score_)
        best_params_list.append(grid_search.best_params_)

        y_test_pred_proba = model_svm_rbf.predict_proba(X_test_normalized)[:, 1]  
        thresholds = np.arange(0.01, 1.0, 0.01)
        best_threshold = max(thresholds, key=lambda t: balanced_accuracy_score(y_test, (y_test_pred_proba > t).astype(int)))

        if len(np.unique(y_test)) > 1:
            auc_score = roc_auc_score(y_test, y_test_pred_proba)
            auprc = average_precision_score(y_test, y_test_pred_proba)
            auc_scores.append(auc_score)
            auprc_scores.append(auprc)
            print(f"AUC Score: {auc_score}")
            print(f"AUPRC Score: {auprc}")
        else:
            auc_scores.append(None)
            auprc_scores.append(None)
            print(f"Skipping AUC and AUPRC computation for this fold as y_test contains only one class: {np.unique(y_test)}")

        y_test_pred_binary = (y_test_pred_proba > best_threshold).astype(int)
        balanced_acc = balanced_accuracy_score(y_test, y_test_pred_binary)
        balanced_accs.append(balanced_acc)
        best_thresholds.append(best_threshold)
        
        print(f"Balanced Accuracy: {balanced_acc}")
        print(f"Best Threshold: {best_threshold}")

        pbar.update(1)

LOSO CV Progress:   0%|                                    | 0/46 [00:00<?]   0%

Train shape: (707, 12), (707,)
Normalized shapes: Train (707, 12), Test (25, 12)
Performing Grid Search for SVM


LOSO CV Progress:   2%|▋                               | 1/46 [00:01<01:13]   2%

Best parameters: {'C': 10, 'class_weight': {0: 1.0, 1: 1.0}, 'gamma': 0.01}
Best cross-validation score: 0.5449295774647888
AUC Score: 0.6602564102564102
AUPRC Score: 0.6539527704572954
Balanced Accuracy: 0.7532051282051282
Best Threshold: 0.48
Train shape: (718, 12), (718,)
Normalized shapes: Train (718, 12), Test (14, 12)
Performing Grid Search for SVM


LOSO CV Progress:   4%|█▎                              | 2/46 [00:02<00:52]   4%

...

Best parameters: {'C': 10, 'class_weight': {0: 1.0, 1: 1.0}, 'gamma': 0.01}
Best cross-validation score: 0.5449899153374351
AUC Score: 0.7563025210084034
AUPRC Score: 0.8775800347204741
Balanced Accuracy: 0.726890756302521
Best Threshold: 0.41000000000000003
Train shape: (706, 12), (706,)
Normalized shapes: Train (706, 12), Test (26, 12)
Performing Grid Search for SVM


LOSO CV Progress:  98%|█████████████████████████████▎ | 45/46 [00:42<00:00]  98%

Best parameters: {'C': 1, 'class_weight': {0: 1.0, 1: 1.0}, 'gamma': 0.01}
Best cross-validation score: 0.5601483781918566
AUC Score: 0.6583333333333333
AUPRC Score: 0.8501505103943185
Balanced Accuracy: 0.6583333333333333
Best Threshold: 0.46
Train shape: (678, 12), (678,)
Normalized shapes: Train (678, 12), Test (54, 12)
Performing Grid Search for SVM


LOSO CV Progress: 100%|██████████████████████████████ | 46/46 [00:43<00:00] 100%

Best parameters: {'C': 10, 'class_weight': {0: 1.0, 1: 1.0}, 'gamma': 0.01}
Best cross-validation score: 0.5352772887323943
AUC Score: 0.6849894291754757
AUPRC Score: 0.4987332603358893
Balanced Accuracy: 0.6775898520084567
Best Threshold: 0.5


In [15]:
print('SVM with Radial basis function (RBF) kernel')
print('Median, 25th Percentile, 75th Percentile: ')

if auc_scores:
    auc_scores_valid = [score for score in auc_scores if score is not None] 

    auc_median = np.median(auc_scores_valid)
    auc_25_percentile = np.percentile(auc_scores_valid, 25)
    auc_75_percentile = np.percentile(auc_scores_valid, 75)
    print(f"AUC Score - Median: {auc_median}, 25th Percentile: {auc_25_percentile}, 75th Percentile: {auc_75_percentile}")

if auprc_scores:
    auprc_scores_valid = [score for score in auprc_scores if score is not None] 

    auprc_median = np.median(auprc_scores_valid)
    auprc_25_percentile = np.percentile(auprc_scores_valid, 25)
    auprc_75_percentile = np.percentile(auprc_scores_valid, 75)
    print(f"AUPRC Score - Median: {auprc_median}, 25th Percentile: {auprc_25_percentile}, 75th Percentile: {auprc_75_percentile}")

balanced_acc_median = np.median(balanced_accs)
balanced_acc_25_percentile = np.percentile(balanced_accs, 25)
balanced_acc_75_percentile = np.percentile(balanced_accs, 75)

print(f"Balanced Accuracy - Median: {balanced_acc_median}, 25th Percentile: {balanced_acc_25_percentile}, 75th Percentile: {balanced_acc_75_percentile}")

SVM with Radial basis function (RBF) kernel
Median, 25th Percentile, 75th Percentile: 
AUC Score - Median: 0.6338935574229692, 25th Percentile: 0.3916666666666666, 75th Percentile: 0.7547268907563026
AUPRC Score - Median: 0.7263643836677545, 25th Percentile: 0.5254744557593822, 75th Percentile: 0.8791545030405508
Balanced Accuracy - Median: 0.714985994397759, 25th Percentile: 0.6, 75th Percentile: 0.825
